In [1]:
import pandas as pd
import numpy as np

In [2]:
users = pd.read_csv('users.dat',  header=None)

In [3]:
raw_data = pd.read_csv('raw-data.csv', encoding='iso-8859-1' );

In [4]:
article_df = pd.DataFrame(raw_data, columns = ['doc.id', 'raw.title', 'raw.abstract'])

In [5]:
article_df = article_df.rename(index=str, columns={"doc.id": "ArticleID", "raw.title": "Title", "raw.abstract":"Abstract"})

In [6]:
users[0] = users[0].str.split(' ')   

In [7]:
ratings = pd.read_csv('ratings.csv')

In [8]:
ratings

,userId,articleId,rating


In [9]:
def fillRatings(ratings):
    columns = ['userId','articleId', 'rating']
    for index, user in enumerate(users[0]):
        for article in user:
            df = pd.DataFrame([[index, article,5.0], [index, article,5.0]], columns=columns) 
            frames = [ratings, df]
            ratings = pd.concat(frames, ignore_index=True)
    return ratings

In [10]:
new_rate = fillRatings(ratings)

In [20]:
ax = new_rate.drop_duplicates()

In [39]:
ratings_df =  pd.read_csv('ratings.csv')

In [40]:
ratings_df = ratings_df.append(ax, ignore_index=True)
ratings_df=ratings_df.rename(columns = {'userId':'UserID', 'articleId':'ArticleID','rating':'Rating' })

In [41]:
article_df['ArticleID'] = article_df['ArticleID'].apply(pd.to_numeric)

In [42]:
article_df.head()

,ArticleID,Title,Abstract
0,1,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...
1,2,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...
2,3,Exploring complex networks,"The study of networks pervades all of science,..."
3,4,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...
4,5,Navigation in a small world,The small-world phenomenon â the principle t...


In [43]:
ratings_df.head()

,UserID,ArticleID,Rating
0,0,70,5.0
1,0,495,5.0
2,0,1631,5.0
3,0,2317,5.0
4,0,2526,5.0


In [44]:
R_df = ratings_df.pivot(index = 'UserID', columns ='ArticleID', values = 'Rating').fillna(0)

In [45]:
R_df.head()

ArticleID,0,1,10,100,1000,10000,10001,10002,10003,10004,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
UserID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [47]:
from scipy.sparse.linalg import svds

In [48]:
U, sigma, Vt = svds(R_demeaned, k = 50)

In [49]:
sigma = np.diag(sigma)

In [50]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [66]:
def recommend_article(predictions_df, userID, article_df, original_ratings_df, num_recommendations=1):
    
    # Pega e classifica as predições do usuário
    user_row_number = userID - 1 # Assumindo que o ID do usuário se inicia em 1
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Pega os dados do usuário e mescla com as informções do artigo.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(article_df, how = 'left', left_on = 'ArticleID', right_on = 'ArticleID').
                     sort_values(['Rating'], ascending=False)
                 )
    
    # Recomenda os artigos melhores avaliados e que não foram lidos pelo usuário
    recommendations = (article_df[~article_df['ArticleID'].isin(user_full['ArticleID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'ArticleID',
               right_on = 'ArticleID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations


In [ ]:
for index, user in enumerate(users[0]):
    already_rated, predictions = recommend_article(preds_df, index+1, article_df, ratings_df, 1)
    print('The recommendation for the user', index, 'is the article:', predictions)

    

Usuário 1 já leu 39 artigos.
The recommendation for the user 0 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2 já leu 21 artigos.
The recommendation for the user 1 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 3 já leu 13 artigos.
The recommendation for the user 2 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 4 já leu 13 artigos.
The r

The recommendation for the user 28 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 30 já leu 34 artigos.
The recommendation for the user 29 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 31 já leu 15 artigos.
The recommendation for the user 30 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 32 já leu 17 artigos.
The recommendation for the u

Usuário 57 já leu 68 artigos.
The recommendation for the user 56 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 58 já leu 38 artigos.
The recommendation for the user 57 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 59 já leu 28 artigos.
The recommendation for the user 58 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 60 já leu 12 artigos

The recommendation for the user 83 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 85 já leu 15 artigos.
The recommendation for the user 84 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 86 já leu 53 artigos.
The recommendation for the user 85 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 87 já leu 35 artigos.
The recommendation for the u

The recommendation for the user 110 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 112 já leu 118 artigos.
The recommendation for the user 111 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 113 já leu 24 artigos.
The recommendation for the user 112 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 114 já leu 45 artigos.
The recommendation fo

...

The recommendation for the user 2106 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2108 já leu 11 artigos.
The recommendation for the user 2107 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2109 já leu 186 artigos.
The recommendation for the user 2108 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2110 já leu 199 artigos.
The recommenda

Usuário 2133 já leu 90 artigos.
The recommendation for the user 2132 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2134 já leu 39 artigos.
The recommendation for the user 2133 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2135 já leu 12 artigos.
The recommendation for the user 2134 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2136 já 

Usuário 2158 já leu 20 artigos.
The recommendation for the user 2157 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2159 já leu 58 artigos.
The recommendation for the user 2158 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2160 já leu 74 artigos.
The recommendation for the user 2159 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2161 já 

Usuário 2184 já leu 95 artigos.
The recommendation for the user 2183 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2185 já leu 16 artigos.
The recommendation for the user 2184 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2186 já leu 27 artigos.
The recommendation for the user 2185 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2187 já 

Usuário 2209 já leu 39 artigos.
The recommendation for the user 2208 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2210 já leu 13 artigos.
The recommendation for the user 2209 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2211 já leu 20 artigos.
The recommendation for the user 2210 is the article:   ArticleID                                              Title  \
0         1  The metabolic world of Escherichia coli is not...   

                                            Abstract  
0  To elucidate the organizational and evolutiona...  
Usuário 2212 já 